# Price Optimization for Hotels

In [1]:
pip install --upgrade pip

     |████████████████████████████████| 2.1MB 3.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Getting snowflake related packages installed and imported

In [2]:
pip install "snowflake-connector-python[pandas]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 187.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.7/444.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 296.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 157.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 271.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 226.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 249.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 335.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 264.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforio 1.0.1 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
mlflow 2.10.0 requires packaging<24, but you have packaging 24.1 which is incompatible.
mlflow 2.10.0 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.1.0 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
refractio 2.1.5.4 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
jupyterlab 3.2.4 requires jupyter-server~=1.4, but you have jupyter-server 2.0.0a1 which is incompatible.
jupyterlab-server 2.25.4 requires jsonschema>=4.18.0, but you have jsonschema 3.2.0 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-serving 1.0.0 requires Flask==2.1.1; python_version >= 

In [3]:
pip install --upgrade --q snowflake-snowpark-python==1.9.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.10.0 requires packaging<24, but you have packaging 24.1 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
jupyterlab 3.2.4 requires jupyter-server~=1.4, but you have jupyter-server 2.0.0a1 which is incompatible.
jupyterlab-server 2.25.4 requires jsonschema>=4.18.0, but you have jsonschema 3.2.0 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-serving 1.0.0 requires Flask==2.1.1; python_version >= "3.7", but you have flask 2.3.3 which is incompatible.
mosaic-ai-serving 1.0.0 requires itsdangerous==2.0.1, but you have itsdangerous 2.2.0 which is incompatible.
mosaic-ai-serving 1.0.0 requires Jinja2==3.0.3, but you have jinja2 3.1.4 which is incompatible.
mosaic-ai-se

In [4]:
pip install fosforio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 237.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 221.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 323.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 316.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 332.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
snowflake-connector-python 3.11.0 requires urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you have urllib3 2.2.1 which is incompatible.
snowflake-snowpark-python 1.9.0 requires cloudpickle<=2.0.0,>=1.6.0; python_version < "3.11", but you have cloudpickle 2.2.1 which is incompatible.
jupyterlab 3.2.4 requires jupyter-

In [5]:
pip install fosforml

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 219.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 202.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.6/736.6 kB 344.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 241.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 265.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 280.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 55.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 274.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 231.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 330.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 29

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 59.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.0.1 requires cloudpickle==3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1; python_version >= "3.8", but you have scikit-learn 1.3.2 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-client 1.0.0 requires requests-toolbelt==0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.
mosaic-ai-serving 1.0.0 requires Flask==2.1.1; python_version >= "3.7", but you have flask 2.3.3 which is incompatible.
mosaic-ai-serving 1.0.0 requires itsdangerous==2.0.1, but you have itsdangerous 2.2.0 which is incompatible.
mosaic-ai-serving 1.0.0 requires Jinja2==3.0.3, but you ha

In [7]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 258.4 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 278.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 270.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 342.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 320.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforio 1.0.1 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
mlflow 2.10.0 requires pytz<2024, but you have pytz 2024.1 which is incompatible.
refractio 2.1.5.4 requires pandas==2.0.0, but you have pandas 2.0.3 which is incompatible.
fosforml 1.0.1 requires cloudpickle==3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1;

In [8]:
pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 110.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 336.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 268.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 318.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 171.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 233.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 189.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 289.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 320.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.10.

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 193.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 330.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 238.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 217.7 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.0.1 requires cloudpickle==3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1; python_version >= "3.8", but you have scikit-learn 1.3.2 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-client 1.0.0 requires requests-toolbelt==0.9.1, but you h

In [10]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 27.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 38.5 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.0.1 requires cloudpickle==3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1; python_version >= "3.8", but you have scikit-learn 1.3.2 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-client 1.0.0 requires requests-toolbelt==0.9.

In [11]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 170.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 243.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 235.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 323.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 179.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 249.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 174.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 197.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 157.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 204.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install python-dateutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 6.4 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 3.2.4 requires jupyter-server~=1.4, but you have jupyter-server 2.0.0a1 which is incompatible.
jupyterlab-server 2.25.4 requires jsonschema>=4.18.0, but you have jsonschema 3.2.0 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.7.5 which is incompatible.
mosaic-ai-client 1.0.0 requires requests-toolbelt==0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.
mosaic-ai-serving 1.0.0 requires Flask==2.1.1; python_version >= "3.7", but you have flask 2.3.3 which is incompatible.
mosaic-ai-serving 1.0.0 requires itsdangerous==2.0.1, but you have itsdangerous 2.2.0 which is incompatible.
mosaic-ai-serving 1.0.0 requires Jinja2==3.0.3, but you have jinja2 3.1.

In [13]:
pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Restart kernal and continue

# Importing packages

In [1]:
from snowflake.snowpark import Session

In [2]:
from snowflake.connector.pandas_tools import write_pandas

In [3]:
from fosforio import snowflake

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import numpy as np
import pandas as pd

In [6]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error

Matplotlib created a temporary cache directory at /tmp/matplotlib-22g3f2wh because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [7]:
from joblib import dump, load
import requests
from tqdm import tqdm
import time
from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime

# Getting data from snowflake

In [8]:
# To get snowflake connection object with a default snowflake connection created by the user, if available.
snowflake.get_connection("TTH_REV_OPT_CXN")

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f7b124c1fa0>
Please close the connection after use!


In [79]:
df = snowflake.get_dataframe("BOOKINGS_TRANSFORMED")
df['ADR']= round(df['ADR'])

In [80]:
df.shape

(109711, 34)

# EDA

In [81]:
# Removing cancelled transactions to model based on transactions that customers preferred to actually complete
df0 = df[(df['IS_CANCELED'] == 0) & (df['RESERVATION_STATUS'] !='No-Show')] 
df0

,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
2388,2021-07-01,2021-07-02,Resort Hotel,0,0,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,2,0,0,2,1,1,BB,FRA,Corporate,Corporate,0,0,A,G,No Deposit,0,Transient,107.0
2410,2021-07-01,2021-07-02,Resort Hotel,0,1,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,PRT,Corporate,Corporate,0,4,A,B,No Deposit,0,Transient,134.0
2425,2021-07-01,2021-07-02,Resort Hotel,0,2,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,PRT,Online TA,TA/TO,0,1,A,C,No Deposit,0,Transient,4.0
2427,2021-07-01,2021-07-02,Resort Hotel,0,4,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,PRT,Corporate,Corporate,0,4,A,C,No Deposit,0,Transient,134.0
2561,2021-07-01,2021-07-02,Resort Hotel,0,13,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,GBR,Corporate,Corporate,0,0,A,A,No Deposit,0,Transient,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109706,2023-08-24,2023-09-10,Resort Hotel,0,269,2023,8,34,24,24-08-2023,Check.Out,10-09-2023,17,17.0,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,85.0
109707,2023-08-31,2023-09-10,Resort Hotel,0,212,2023,8,35,31,31-08-2023,Check.Out,10-09-2023,10,10.0,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,90.0
109708,2023-08-29,2023-09-12,Resort Hotel,0,204,2023,8,35,29,29-08-2023,Check.Out,12-09-2023,14,14.0,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,154.0
109709,2023-08-31,2023-09-14,Resort Hotel,0,161,2023,8,35,31,31-08-2023,Check.Out,14-09-2023,14,14.0,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.0


In [82]:
#Understanding what market segments need to be considered
df0.groupby(['HOTEL','RESERVED_ROOM_TYPE','MARKET_SEGMENT']).agg({'ADR':'mean','RESERVATION_STATUS_DATE_TRANSFORMED':'count'})

ADR  \
HOTEL        RESERVED_ROOM_TYPE MARKET_SEGMENT               
City Hotel   A                  Aviation         96.505155   
                                Complementary    45.818182   
                                Corporate        80.057859   
                                Direct          107.746743   
                                Groups           86.905615   
                                Offline TA/TO    91.279886   
                                Online TA       106.459990   
             D                  Aviation        107.716049   
                                Complementary    25.000000   
                                Corporate       100.673913   
                                Direct          132.383333   
                                Groups           87.323529   
                                Offline TA/TO    90.285714   
                                Online TA       136.453288   
             E                  Aviation        159.500000   
                                Complementary    55.000000   
                                Corporate       114.795455   
                                Direct          164.451852   
                                Groups           80.000000   
                                Offline TA/TO   133.465753   
                                Online TA       169.339578   
Resort Hotel A                  Complementary    21.230769   
                                Corporate        49.697072   
                                Direct           84.609010   
                                Groups           67.293482   
                                Offline TA/TO    64.203886   
                                Online TA        90.872148   
             D                  Complementary    41.333333   
                                Corporate        67.391304   
                                Direct          116.984472   
                                Groups           74.854637   
                                Offline TA/TO    85.349315   
                                Online TA       113.250907   
             E                  Complementary     7.333333   
                                Corporate        61.509091   
                                Direct          118.988764   
                                Groups           87.105991   
                                Offline TA/TO    97.495172   
                                Online TA       117.959196   

                                                RESERVATION_STATUS_DATE_TRANSFORMED  
HOTEL        RESERVED_ROOM_TYPE MARKET_SEGMENT                                       
City Hotel   A                  Aviation                                         97  
                                Complementary                                    22  
                                Corporate                                      2074  
                                Direct                                         3147  
                                Groups                                         4132  
                                Offline TA/TO                                  8457  
                                Online TA                                     16496  
             D                  Aviation                                         81  
                                Complementary                                     4  
                                Corporate                                       138  
                                Direct                                          900  
                                Groups                                           68  
                                Offline TA/TO                                   735  
                                Online TA                                      5566  
             E                  Aviation                                          2  
                                Complementary     

#We will filter for room _types A,D,E and will filter OUT "Complementary" market_segment. retain both City and resort hotel types.
##"For our pricing model, we need to count the frequency of each price point

##Moreover, Hotels have different room types with different prices. To create our pricing model, we need to optimize the room type level. We are interested in the room type reserved as this captures the price the guest was willing to pay for that room type. It is common for hotels to overbook the lower room types and give free upgrades to higher room types. Therefore, any pricing analysis must be done at the reserved room type and not the stayed room type.

##Unfortunately, there is not much description of the reserved room type, but by looking at the price points, we can infer that A &B are the lowest price points. The rest of the room types command a higher ADR, which means they likely are upgraded rooms. In another workbook, we will explore how to price those premium rooms, but for the moment, let's focus on room types A & D, E.

#filter data to only Direct and Online TA and room typ A&B
##Create individual datasets for City, Resort, room types: A,D,E

In [83]:
data = df0[(df.MARKET_SEGMENT != 'Complementary') & (df.MARKET_SEGMENT != 'Corporate') & (df.MARKET_SEGMENT !='Aviation')]
data = data[(data.RESERVED_ROOM_TYPE == 'A') |(data.RESERVED_ROOM_TYPE == 'D') | (data.RESERVED_ROOM_TYPE == 'E')]
data.RESERVED_ROOM_TYPE.value_counts()

RESERVED_ROOM_TYPE
A    46925
D    12590
E     4341
Name: count, dtype: int64

In [84]:
#Understanding what market segments need to be considered
data.groupby(['HOTEL','RESERVED_ROOM_TYPE','MARKET_SEGMENT']).agg({'ADR':'mean','RESERVATION_STATUS_DATE_TRANSFORMED':'count'})

ADR  \
HOTEL        RESERVED_ROOM_TYPE MARKET_SEGMENT               
City Hotel   A                  Direct          107.746743   
                                Groups           86.905615   
                                Offline TA/TO    91.279886   
                                Online TA       106.459990   
             D                  Direct          132.383333   
                                Groups           87.323529   
                                Offline TA/TO    90.285714   
                                Online TA       136.453288   
             E                  Direct          164.451852   
                                Groups           80.000000   
                                Offline TA/TO   133.465753   
                                Online TA       169.339578   
Resort Hotel A                  Direct           84.609010   
                                Groups           67.293482   
                                Offline TA/TO    64.203886   
                                Online TA        90.872148   
             D                  Direct          116.984472   
                                Groups           74.854637   
                                Offline TA/TO    85.349315   
                                Online TA       113.250907   
             E                  Direct          118.988764   
                                Groups           87.105991   
                                Offline TA/TO    97.495172   
                                Online TA       117.959196   

                                                RESERVATION_STATUS_DATE_TRANSFORMED  
HOTEL        RESERVED_ROOM_TYPE MARKET_SEGMENT                                       
City Hotel   A                  Direct                                         3147  
                                Groups                                         4132  
                                Offline TA/TO                                  8457  
                                Online TA                                     16496  
             D                  Direct                                          900  
                                Groups                                           68  
                                Offline TA/TO                                   735  
                                Online TA                                      5566  
             E                  Direct                                          405  
                                Groups                                            2  
                                Offline TA/TO                                    73  
                                Online TA                                       427  
Resort Hotel A                  Direct                                         2486  
                                Groups                                         2593  
                                Offline TA/TO                                  3654  
                                Online TA                                      5960  
             D                  Direct                                          966  
                                Groups                                          399  
                                Offline TA/TO                                  1752  
                                Online TA                                      2204  
             E                  Direct                                          801  
                                Groups                                          217  
                                Offline TA/TO                                   725  
                                Online TA                                      1691

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63856 entries, 2425 to 109710
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ARRIVAL_DATE_TRANSFORMED             63856 non-null  object 
 1   RESERVATION_STATUS_DATE_TRANSFORMED  63856 non-null  object 
 2   HOTEL                                63856 non-null  object 
 3   IS_CANCELED                          63856 non-null  int8   
 4   LEAD_TIME                            63856 non-null  int16  
 5   ARRIVAL_DATE_YEAR                    63856 non-null  int16  
 6   MONTH                                63856 non-null  int8   
 7   ARRIVAL_DATE_WEEK_NUMBER             63856 non-null  int8   
 8   ARRIVAL_DATE_DAY_OF_MONTH            63856 non-null  int8   
 9   EXPECTED_ARRIVAL_DATE                63856 non-null  object 
 10  RESERVATION_STATUS                   63856 non-null  object 
 11  RESERVATION_STATUS_DATE      

In [86]:
# checking num dates
data.groupby(['ARRIVAL_DATE_TRANSFORMED']).agg({'ADR':'mean','RESERVATION_STATUS_DATE_TRANSFORMED':'count'})

,ADR,RESERVATION_STATUS_DATE_TRANSFORMED
ARRIVAL_DATE_TRANSFORMED,,
2021-07-01,93.366667,90
2021-07-02,92.275862,29
2021-07-03,94.156250,32
2021-07-04,88.857143,42
2021-07-05,95.000000,30
...,...,...
2023-08-27,130.818182,110
2023-08-28,134.835821,134
2023-08-29,141.671642,67


In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63856 entries, 2425 to 109710
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ARRIVAL_DATE_TRANSFORMED             63856 non-null  object 
 1   RESERVATION_STATUS_DATE_TRANSFORMED  63856 non-null  object 
 2   HOTEL                                63856 non-null  object 
 3   IS_CANCELED                          63856 non-null  int8   
 4   LEAD_TIME                            63856 non-null  int16  
 5   ARRIVAL_DATE_YEAR                    63856 non-null  int16  
 6   MONTH                                63856 non-null  int8   
 7   ARRIVAL_DATE_WEEK_NUMBER             63856 non-null  int8   
 8   ARRIVAL_DATE_DAY_OF_MONTH            63856 non-null  int8   
 9   EXPECTED_ARRIVAL_DATE                63856 non-null  object 
 10  RESERVATION_STATUS                   63856 non-null  object 
 11  RESERVATION_STATUS_DATE      

In [88]:
data.head(10)

,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
2425,2021-07-01,2021-07-02,Resort Hotel,0,2,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,PRT,Online TA,TA/TO,0,1,A,C,No Deposit,0,Transient,4.0
2652,2021-07-01,2021-07-02,Resort Hotel,0,7,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,1,0,0,1,1,1,BB,GBR,Direct,Direct,0,0,A,C,No Deposit,0,Transient,75.0
2678,2021-07-01,2021-07-02,Resort Hotel,0,12,2021,7,27,1,01-07-2021,Check.Out,02-07-2021,1,1.0,0,1,2,0,0,2,1,1,BB,IRL,Online TA,TA/TO,0,0,A,E,No Deposit,0,Transient,88.0
2757,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,1,0,0,1,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,80.0
2759,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0
2763,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0
2765,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0
2769,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0
2770,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0
2771,2021-07-01,2021-07-03,City Hotel,0,257,2021,7,27,1,01-07-2021,Check.Out,03-07-2021,2,2.0,0,2,2,0,0,2,1,2,HB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,102.0


In [89]:
# drop unrequired columns
data1 = data.drop(['TALLY_DAYS', 'DEPOSIT_TYPE', 'ASSIGNED_ROOM_TYPE','IS_CANCELED','ADULTS', 'CHILDREN', 'BABIES', 'AVG_ROOMS_PER_NIGHT', 'MEAL', 'COUNTRY', 'DISTRIBUTION_CHANNEL', 'PREVIOUS_BOOKINGS_NOT_CANCELED','DEPOSIT_TYPE','DAYS_IN_WAITING_LIST', 'RESERVATION_STATUS_DATE', 'PREVIOUS_CANCELLATIONS','EXPECTED_ARRIVAL_DATE','LEAD_TIME','TOTAL_ROOM_NIGHTS'], axis=1)
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63856 entries, 2425 to 109710
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ARRIVAL_DATE_TRANSFORMED             63856 non-null  object 
 1   RESERVATION_STATUS_DATE_TRANSFORMED  63856 non-null  object 
 2   HOTEL                                63856 non-null  object 
 3   ARRIVAL_DATE_YEAR                    63856 non-null  int16  
 4   MONTH                                63856 non-null  int8   
 5   ARRIVAL_DATE_WEEK_NUMBER             63856 non-null  int8   
 6   ARRIVAL_DATE_DAY_OF_MONTH            63856 non-null  int8   
 7   RESERVATION_STATUS                   63856 non-null  object 
 8   TOTAL_STAY_NIGHTS                    63856 non-null  int8   
 9   STAYS_IN_WEEKEND_NIGHTS              63856 non-null  int8   
 10  STAYS_IN_WEEK_NIGHTS                 63856 non-null  int8   
 11  TOTAL_GUESTS                 

In [90]:
# renaming some column names
data1.rename(columns = {'ARRIVAL_DATE_TRANSFORMED':'arrival_date', 'RESERVATION_STATUS_DATE_TRANSFORMED':'reservation_date',
                        'HOTEL':'hotel', 'ARRIVAL_DATE_YEAR':'year','MONTH':'month', 
                        'ARRIVAL_DATE_WEEK_NUMBER': 'week', 'ARRIVAL_DATE_DAY_OF_MONTH': 'dom', 
                        'RESERVATION_STATUS': 'reservation_status', 'TOTAL_STAY_NIGHTS': 'total_rns',
                        'STAYS_IN_WEEKEND_NIGHTS':'weekend_nights', 
                        'STAYS_IN_WEEK_NIGHTS': 'week_nights','TOTAL_GUESTS': 'total_guests','MARKET_SEGMENT':'market_segment', 
                        'RESERVED_ROOM_TYPE': 'room_type', 'CUSTOMER_TYPE':'cust_type', 'ADR':'adr'}, inplace = True)

In [91]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63856 entries, 2425 to 109710
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        63856 non-null  object 
 1   reservation_date    63856 non-null  object 
 2   hotel               63856 non-null  object 
 3   year                63856 non-null  int16  
 4   month               63856 non-null  int8   
 5   week                63856 non-null  int8   
 6   dom                 63856 non-null  int8   
 7   reservation_status  63856 non-null  object 
 8   total_rns           63856 non-null  int8   
 9   weekend_nights      63856 non-null  int8   
 10  week_nights         63856 non-null  int8   
 11  total_guests        63856 non-null  int8   
 12  market_segment      63856 non-null  object 
 13  room_type           63856 non-null  object 
 14  cust_type           63856 non-null  object 
 15  adr                 63856 non-null  float64
dtypes: fl

In [92]:
# checking num dates
data1.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-07-01,93.366667,90
2021-07-02,92.275862,29
2021-07-03,94.156250,32
2021-07-04,88.857143,42
2021-07-05,95.000000,30
...,...,...
2023-08-27,130.818182,110
2023-08-28,134.835821,134
2023-08-29,141.671642,67


In [93]:
# Creating separate tables for Optimizing models
ca = data1[(data1['hotel'] == 'City Hotel') & (data1['room_type'] == 'A')]
cd = data1[(data1['hotel'] == 'City Hotel') & (data1['room_type'] == 'D')]
ce = data1[(data1['hotel'] == 'City Hotel') & (data1['room_type'] == 'E')]
ra = data1[(data1['hotel'] == 'Resort Hotel') & (data1['room_type'] == 'A')]
rd = data1[(data1['hotel'] == 'Resort Hotel') & (data1['room_type'] == 'D')]
re = data1[(data1['hotel'] == 'Resort Hotel') & (data1['room_type'] == 'E')]

In [94]:
ca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32232 entries, 2757 to 109679
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        32232 non-null  object 
 1   reservation_date    32232 non-null  object 
 2   hotel               32232 non-null  object 
 3   year                32232 non-null  int16  
 4   month               32232 non-null  int8   
 5   week                32232 non-null  int8   
 6   dom                 32232 non-null  int8   
 7   reservation_status  32232 non-null  object 
 8   total_rns           32232 non-null  int8   
 9   weekend_nights      32232 non-null  int8   
 10  week_nights         32232 non-null  int8   
 11  total_guests        32232 non-null  int8   
 12  market_segment      32232 non-null  object 
 13  room_type           32232 non-null  object 
 14  cust_type           32232 non-null  object 
 15  adr                 32232 non-null  float64
dtypes: fl

In [95]:
# checking num dates
ca.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-07-01,96.937500,64
2021-07-02,59.000000,1
2021-07-03,74.500000,10
2021-07-04,63.777778,9
2021-07-06,66.000000,1
...,...,...
2023-08-27,103.500000,54
2023-08-28,104.935065,77
2023-08-29,118.857143,35


In [96]:
cd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7269 entries, 5973 to 109688
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        7269 non-null   object 
 1   reservation_date    7269 non-null   object 
 2   hotel               7269 non-null   object 
 3   year                7269 non-null   int16  
 4   month               7269 non-null   int8   
 5   week                7269 non-null   int8   
 6   dom                 7269 non-null   int8   
 7   reservation_status  7269 non-null   object 
 8   total_rns           7269 non-null   int8   
 9   weekend_nights      7269 non-null   int8   
 10  week_nights         7269 non-null   int8   
 11  total_guests        7269 non-null   int8   
 12  market_segment      7269 non-null   object 
 13  room_type           7269 non-null   object 
 14  cust_type           7269 non-null   object 
 15  adr                 7269 non-null   float64
dtypes: flo

In [97]:
# checking num dates
cd.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-08-05,75.000000,1
2021-08-08,100.000000,2
2021-08-11,15.000000,1
2021-08-12,105.000000,1
2021-08-13,58.000000,1
...,...,...
2023-08-27,144.200000,15
2023-08-28,142.800000,20
2023-08-29,162.636364,11


In [98]:
ce.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 6313 to 109693
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        907 non-null    object 
 1   reservation_date    907 non-null    object 
 2   hotel               907 non-null    object 
 3   year                907 non-null    int16  
 4   month               907 non-null    int8   
 5   week                907 non-null    int8   
 6   dom                 907 non-null    int8   
 7   reservation_status  907 non-null    object 
 8   total_rns           907 non-null    int8   
 9   weekend_nights      907 non-null    int8   
 10  week_nights         907 non-null    int8   
 11  total_guests        907 non-null    int8   
 12  market_segment      907 non-null    object 
 13  room_type           907 non-null    object 
 14  cust_type           907 non-null    object 
 15  adr                 907 non-null    float64
dtypes: floa

In [99]:
# checking num dates
ce.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-08-07,60.00,1
2021-08-09,68.00,2
2021-08-11,95.00,1
2021-08-19,89.00,1
2021-08-26,148.00,1
...,...,...
2023-08-27,204.75,4
2023-08-28,195.00,3
2023-08-29,209.00,1


In [100]:
ra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14693 entries, 2425 to 109709
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        14693 non-null  object 
 1   reservation_date    14693 non-null  object 
 2   hotel               14693 non-null  object 
 3   year                14693 non-null  int16  
 4   month               14693 non-null  int8   
 5   week                14693 non-null  int8   
 6   dom                 14693 non-null  int8   
 7   reservation_status  14693 non-null  object 
 8   total_rns           14693 non-null  int8   
 9   weekend_nights      14693 non-null  int8   
 10  week_nights         14693 non-null  int8   
 11  total_guests        14693 non-null  int8   
 12  market_segment      14693 non-null  object 
 13  room_type           14693 non-null  object 
 14  cust_type           14693 non-null  object 
 15  adr                 14693 non-null  float64
dtypes: fl

In [101]:
# checking num dates
ra.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-07-01,74.214286,14
2021-07-02,77.000000,15
2021-07-03,94.928571,14
2021-07-04,83.000000,18
2021-07-05,90.714286,21
...,...,...
2023-08-27,150.052632,19
2023-08-28,185.692308,13
2023-08-29,165.000000,9


In [102]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5321 entries, 2916 to 109710
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        5321 non-null   object 
 1   reservation_date    5321 non-null   object 
 2   hotel               5321 non-null   object 
 3   year                5321 non-null   int16  
 4   month               5321 non-null   int8   
 5   week                5321 non-null   int8   
 6   dom                 5321 non-null   int8   
 7   reservation_status  5321 non-null   object 
 8   total_rns           5321 non-null   int8   
 9   weekend_nights      5321 non-null   int8   
 10  week_nights         5321 non-null   int8   
 11  total_guests        5321 non-null   int8   
 12  market_segment      5321 non-null   object 
 13  room_type           5321 non-null   object 
 14  cust_type           5321 non-null   object 
 15  adr                 5321 non-null   float64
dtypes: flo

In [103]:
# checking num dates
rd.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-07-01,98.555556,9
2021-07-02,93.833333,6
2021-07-03,94.000000,3
2021-07-04,98.000000,9
2021-07-05,102.833333,6
...,...,...
2023-08-27,158.000000,12
2023-08-28,184.285714,14
2023-08-29,137.400000,5


In [104]:
re.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3434 entries, 2910 to 109708
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   arrival_date        3434 non-null   object 
 1   reservation_date    3434 non-null   object 
 2   hotel               3434 non-null   object 
 3   year                3434 non-null   int16  
 4   month               3434 non-null   int8   
 5   week                3434 non-null   int8   
 6   dom                 3434 non-null   int8   
 7   reservation_status  3434 non-null   object 
 8   total_rns           3434 non-null   int8   
 9   weekend_nights      3434 non-null   int8   
 10  week_nights         3434 non-null   int8   
 11  total_guests        3434 non-null   int8   
 12  market_segment      3434 non-null   object 
 13  room_type           3434 non-null   object 
 14  cust_type           3434 non-null   object 
 15  adr                 3434 non-null   float64
dtypes: flo

In [105]:
# checking num dates
re.groupby(['arrival_date']).agg({'adr':'mean','hotel':'count'})

,adr,hotel
arrival_date,,
2021-07-01,91.000000,3
2021-07-02,128.428571,7
2021-07-03,131.400000,5
2021-07-04,130.333333,6
2021-07-05,109.333333,3
...,...,...
2023-08-27,178.666667,6
2023-08-28,221.857143,7
2023-08-29,193.666667,6


# Data transformation to disaggregate dates
# For Full data

In [106]:
data_to_transform = data1[['hotel','room_type','arrival_date','total_rns','adr']]
data_to_transform.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63856 entries, 2425 to 109710
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotel         63856 non-null  object 
 1   room_type     63856 non-null  object 
 2   arrival_date  63856 non-null  object 
 3   total_rns     63856 non-null  int8   
 4   adr           63856 non-null  float64
dtypes: float64(1), int8(1), object(3)
memory usage: 2.5+ MB


In [107]:
# Create a new dataframe to store the data by stay date
expanded_df = pd.DataFrame()

In [109]:
# Iterate over each row in the expected arrival date
##Need to convert with hotel, room type columns
for _, row in data_to_transform.iterrows():
    # Get the number of stay dates for the current booking
    num_stay_dates = row['total_rns']
    
    #Create a row for each stay date
    expanded_booking = pd.DataFrame({\
        'hotel': row['hotel'],
        'room_type' : row ['room_type'],                             
        'arrival_date': pd.date_range(start=row['arrival_date'], periods=num_stay_dates),
        'total_rns': 1,
        'adr': row['adr']
    })                                
    # Append the stay date information to the new dataframe
    expanded_df = pd.concat([expanded_df, expanded_booking], ignore_index=True)  

In [ ]:
# Sort the final dataframe by date
expanded_df = expanded_df.sort_values('arrival_date')
expanded_df = expanded_df.reset_index(drop=True)
expanded_df

In [ ]:
expanded_booking.info()

# Data transformation to dis-aggregate dates- 
# For City Hotel room type A

In [29]:
ca_data_to_transform = ca[['hotel','room_type','arrival_date','total_rns','adr']]
ca_data_to_transform.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32232 entries, 2757 to 109679
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotel         32232 non-null  object 
 1   room_type     32232 non-null  object 
 2   arrival_date  32232 non-null  object 
 3   total_rns     32232 non-null  int8   
 4   adr           32232 non-null  float64
dtypes: float64(1), int8(1), object(3)
memory usage: 1.3+ MB


In [67]:
# Create a new dataframe to store the data by stay date
ca_expanded_df = pd.DataFrame()

In [68]:
# Iterate over each row in the expected arrival date
##Need to convert with hotel, room type columns
for _, row in ca_data_to_transform.iterrows():
    # Get the number of stay dates for the current booking
    num_stay_dates = row['total_rns']
    
    #Create a row for each stay date
    ca_expanded_booking = pd.DataFrame({
        'hotel': row['hotel'],
        'room_type' : row ['room_type'], 
        'arrival_date': pd.date_range(start=row['arrival_date'], periods=num_stay_dates),
        'total_rns': 1,
        'adr': row['adr'] 
    })
    # Append the stay date information to the new dataframe
    ca_expanded_df = pd.concat([ca_expanded_df, ca_expanded_booking], ignore_index=True)  

In [72]:
ca_expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93775 entries, 0 to 93774
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   hotel         93775 non-null  object        
 1   room_type     93775 non-null  object        
 2   arrival_date  93775 non-null  datetime64[ns]
 3   total_rns     93775 non-null  int64         
 4   adr           93775 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.6+ MB


In [73]:
ca_expanded_df.head(10)

,hotel,room_type,arrival_date,total_rns,adr
0,City Hotel,A,2021-07-01,1,80.0
1,City Hotel,A,2021-07-02,1,80.0
2,City Hotel,A,2021-07-01,1,101.5
3,City Hotel,A,2021-07-02,1,101.5
4,City Hotel,A,2021-07-01,1,101.5
5,City Hotel,A,2021-07-02,1,101.5
6,City Hotel,A,2021-07-01,1,101.5
7,City Hotel,A,2021-07-02,1,101.5
8,City Hotel,A,2021-07-01,1,101.5
9,City Hotel,A,2021-07-02,1,101.5


In [76]:
# Sort the final dataframe by date
ca_expanded_df = ca_expanded_df.sort_values('arrival_date')
ca_expanded_df = ca_expanded_df.reset_index(drop=True)
ca_expanded_df

,hotel,room_type,arrival_date,total_rns,adr
0,City Hotel,A,2021-07-01,1,80.0
1,City Hotel,A,2021-07-01,1,102.0
2,City Hotel,A,2021-07-01,1,102.0
3,City Hotel,A,2021-07-01,1,102.0
4,City Hotel,A,2021-07-01,1,102.0
...,...,...,...,...,...
93770,City Hotel,A,2023-09-05,1,104.0
93771,City Hotel,A,2023-09-05,1,96.0
93772,City Hotel,A,2023-09-05,1,88.0
93773,City Hotel,A,2023-09-06,1,151.0


In [78]:
ca_expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93775 entries, 0 to 93774
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   hotel         93775 non-null  object        
 1   room_type     93775 non-null  object        
 2   arrival_date  93775 non-null  datetime64[ns]
 3   total_rns     93775 non-null  int64         
 4   adr           93775 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.6+ MB
